In [1]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /kaggle/working/processed_data.zip
100%|██████████████████████████████████████| 5.62M/5.62M [00:00<00:00, 8.61MB/s]


In [2]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [3]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 29.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 111.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00


In [4]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [5]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [6]:


models_list_3=[
        "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
"FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",

# "bert",

]


In [7]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
# langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [8]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [9]:
all_csv

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [10]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [11]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [12]:
train_data=all_csv


In [13]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [14]:

train_data=train_data.fillna(0)

train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [15]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [16]:
train_data.dtypes

id               object
anger             int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [17]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...,...
2763,2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [18]:
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...
2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [19]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [20]:
train_data['lang'].value_counts()

lang
eng    2768
Name: count, dtype: int64

In [21]:
sum(train_data['lang'].isna())

0

In [22]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [23]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [24]:
train_data[train_data["lang"]=="deu"].dtypes

id               object
anger             int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [25]:
for lang in langs:
    for label in label_columns:
        print(lang,label,train_data[train_data["lang"]=="afr"][label].unique())

eng anger []
eng fear []
eng joy []
eng sadness []
eng surprise []


In [26]:
nan_rows["lang"].value_counts()

Series([], Name: count, dtype: int64)

In [27]:
nan_rows.dtypes

id               object
anger             int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [28]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

              id   fear    joy  sadness  surprise   lang  clean_message
lang                                                                   
eng  0     False  False  False    False     False  False          False
     1     False  False  False    False     False  False          False
     2     False  False  False    False     False  False          False
     3     False  False  False    False     False  False          False
     4     False  False  False    False     False  False          False
...          ...    ...    ...      ...       ...    ...            ...
     2763  False  False  False    False     False  False          False
     2764  False  False  False    False     False  False          False
     2765  False  False  False    False     False  False          False
     2766  False  False  False    False     False  False          False
     2767  False  False  False    False     False  False          False

[2768 rows x 7 columns]


In [30]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [31]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [32]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [33]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [34]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1947      267  2214
fear          943     1271  2214
joy          1661      553  2214
sadness      1513      701  2214
surprise     1552      662  2214


In [35]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [36]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([4.1461, 0.8710, 2.0018, 1.5792, 1.6722], device='cuda:0')

In [37]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [38]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [39]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [40]:
def compute_metrics(p):

  predicted_logits = p.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average="weighted")

  accuracy = accuracy_score(y_valid, predicted_classes)

  return {

        "accuracy": accuracy,

        "precision": precision,

        "recall": recall,

        "f1_score": f1

    }

In [41]:
def evaluate_func(trainer):

  trainer.evaluate()

  predictions = trainer.predict(valid_dataset)

  predicted_logits = predictions.predictions

  print(f"Type of predicted_logits: {type(predicted_logits)}")

  if isinstance(predicted_logits, tuple):

        predicted_logits = predicted_logits[0]

  if not isinstance(predicted_logits, np.ndarray):

        predicted_logits = np.array(predicted_logits)

  if len(predicted_logits.shape) == 3:

        predicted_logits = predicted_logits[:, 0, :]

  predicted_probabilities = expit(predicted_logits)

  threshold = 0.5

  predicted_classes = (predicted_probabilities >= threshold).astype(int)

  precision, recall, f1, _ = precision_recall_fscore_support(y_valid, predicted_classes, average=None,zero_division=0)

  accuracy = accuracy_score(y_valid, predicted_classes)

  return precision, recall, f1,accuracy

In [42]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [43]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [44]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [45]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [46]:
USE_LORA=False

In [47]:


torch.cuda.empty_cache()



print(x)

tokenizer = AutoTokenizer.from_pretrained(x)

model = AutoModelForSequenceClassification.from_pretrained(x, num_labels=len(label_columns),

                                                            ignore_mismatched_sizes=True,

                                                            quantization_config=quantization_config if USE_LORA else None,

                                                            # low_cpu_mem_usage=True,

                                                           torch_dtype=torch.bfloat16,

                                                           problem_type = "multi_label_classification",



                                                            )

FacebookAI/xlm-roberta-large


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(model)

[]

In [49]:
# a=

In [50]:
model.train() # model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  model = prepare_model_for_kbit_training(model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  model = get_peft_model(model, config)



  # trainable parameter count

  model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [51]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if model.config.pad_token_id is None:
        print("setting model pad token")
        model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'<pad>'

In [52]:
device

'cuda'

In [53]:
model = model.to(device)

# for param in model.parameters():

#     param.data = param.data.contiguous()


In [54]:
os.environ['WANDB_DISABLED']="true"
training_args = train_arg()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [55]:
train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH)

In [56]:
# Define a custom loss function to handle class weights for multilabel classification
def custom_loss(output, target):
    # BCEWithLogitsLoss with pos_weight for each class
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
    return loss_fn(output, target)

In [57]:
# Define the Trainer with custom loss function
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [58]:
trainer = CustomTrainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=valid_dataset,

    compute_metrics=compute_metrics,

    # data_collator=data_collator

)

In [59]:
checkpoints = sorted(glob.glob(os.path.join(trainer.args.output_dir, "checkpoint-*")), key=os.path.getmtime)



resume_checkpoint = True if checkpoints else False

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train(resume_from_checkpoint=resume_checkpoint)

model.config.use_cache = True


Step,Training Loss,Validation Loss


In [60]:
model.config.use_cache = True
precision, recall, f1,accuracy=evaluate_func(trainer)
new_data = pd.DataFrame({

  "Models_name": [x],

  "Precision": [precision],

  "Recall": [recall],

  "F1_score": [f1],

  "Accuracy": [accuracy]

})



metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)



model_confusion_matrices = []

predicted_classes = trainer.predict(valid_dataset).predictions







if isinstance(predicted_classes, tuple) :

  predicted_classes = predicted_classes[0]

predicted_classes = np.array(predicted_classes)

# print(f"Type after conversion: {type(predicted_classes)}")



if len(predicted_classes.shape) > 2:

  predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)

  # print(f"Shape after reshaping: {predicted_classes.shape}")









predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)



y_valid_np = np.array(y_valid)



for j in range(len(label_columns)):

    print(f"Computing confusion matrix for label: {label_columns[j]}")

    cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])



    # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])

    model_confusion_matrices.append(cm)

confusion_matrices.append(model_confusion_matrices)





print(f"{x} evaluates:")

for k, label in enumerate(label_columns):

  print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

Type of predicted_logits: <class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of predicted_logits: <class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of predicted_logits: <class 'numpy.ndarray'>


<ipython-input-60-280a069bd3c3>:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)


Type of predicted_logits: <class 'numpy.ndarray'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Computing confusion matrix for label: anger
Computing confusion matrix for label: fear
Computing confusion matrix for label: joy
Computing confusion matrix for label: sadness
Computing confusion matrix for label: surprise
FacebookAI/xlm-roberta-large evaluates:
Class 'anger': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'fear': Precision: 0.6137, Recall: 1.0000, F1: 0.7606
Class 'joy': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'sadness': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Class 'surprise': Precision: 0.0000, Recall: 0.0000, F1: 0.0000
Overall accuracy: 0.1426
-------------------------------------------------------------------------



In [61]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])
# Temporarily disable metric computation
original_compute_metrics = trainer.compute_metrics
trainer.compute_metrics = None

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()
    if(len(lang_valid_data)==0):
      continue
    # Tokenize the validation data
    lang_valid_encodings = tokenizer(lang_x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    lang_valid_dataset = Dataset(lang_valid_encodings, lang_y_valid)

    # Get predictions
    predictions = trainer.predict(lang_valid_dataset,)
    predicted_logits = predictions.predictions

    if isinstance(predicted_logits, tuple):
        predicted_logits = predicted_logits[0]

    if not isinstance(predicted_logits, np.ndarray):
        predicted_logits = np.array(predicted_logits)

    if len(predicted_logits.shape) == 3:
        predicted_logits = predicted_logits[:, 0, :]

    predicted_probabilities = expit(predicted_logits)
    threshold = 0.5
    predicted_classes = (predicted_probabilities >= threshold).astype(int)

    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(lang_y_valid, predicted_classes, average="weighted")
    accuracy = accuracy_score(lang_y_valid, predicted_classes)

    # Store the metrics in the DataFrame
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
        "Accuracy": [accuracy]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)

# Restore compute_metrics function if needed
trainer.compute_metrics = original_compute_metrics

  Language  Precision    Recall  F1_score  Accuracy
0      eng   0.236849  0.385925  0.293545  0.142599


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-61-bf824cdfa0a4>:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)


In [62]:
a=

SyntaxError: invalid syntax (<ipython-input-62-2d920c342ece>, line 1)

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = token

In [ ]:
trainer.push_to_hub("roberta-sentiment")